<h1 style="color: red">Atualizando cotação de moedas na DB</h1>

In [ ]:
# Caso ainda não tenha o selenium instalado basta fazer:
!pip install selenium # instale esse pacote no seu editor de código pelo terminal

In [2]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from datetime import datetime as dt
import pandas as pd

In [ ]:
hoje = dt.now()
momento  = hoje.strftime("%d-%m-%Y %H:%M:%S")

navegador = wd.Chrome()
# Abrindo um navegador Chrome em segundo plano
# chrome_options = Options()
# chrome_options.headless = True 
# navegador = wd.Chrome(options=chrome_options)

# Entrando no página de busca do Google
navegador.get("https://www.google.com/")

# Cotação do Dólar
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
cd = float(cotacao_dolar)
print("O Dólar vale R$ {1:.3f}. Pesquisa feita em: {0}.".format(momento, cd)) # Verificando o preço

# Cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
ce = float(cotacao_euro)
print("O Euro vale R$ {1:.3f}. Pesquisa feita em: {0}.".format(momento, ce)) # Verificando o preço

# Cotação do Ouro
# Acessando em uma página web que tem a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".") # Trocando as vírgulas por pontos (padrão da linguagem)
co = float(cotacao_ouro)
print("O grama do Ouro vale R$ {1:.3f}. Pesquisa feita em: {0}.".format(momento, co)) # Verificando o preço

# Fechando o navegador aberto
navegador.quit()

In [3]:
# Abrindo a DB para manipulação
DB = pd.read_excel("Produtos.xlsx")
display(DB)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [ ]:
# Se a coluna "Moeda" = Dólar, Euro ou Ouro: a coluna "Cotação" é atualizada de acordo com a moeda
DB.loc[DB["Moeda"] == "Dólar", "Cotação"] = round(cd, 3)
DB.loc[DB["Moeda"] == "Euro", "Cotação"] = round(ce, 3)
DB.loc[DB["Moeda"] == "Ouro", "Cotação"] = round(co, 3)

# Atualizando o preço de compra em reais
DB["Preço de Compra"] = DB["Preço Original"] * DB["Cotação"]

# Atualizando o preço de vendas em reais
DB["Preço de Venda"] = DB["Preço de Compra"] * DB["Margem"]

# Formatando cotação em reais
DB["Cotação"] = DB["Cotação"].map("R$ {:.2f}".format)

# Formatando preço de compra em reais
DB["Preço de Compra"] = DB["Preço de Compra"].map("R$ {:.2f}".format)

# Formatando a margem de lucro
DB["Margem"] = DB["Margem"].map("{:.2f} %".format)

# Formatando preço de vendas em reais
DB["Preço de Venda"] = DB["Preço de Venda"].map("R$ {:.2f}".format)

# Formatando preço Original
DB["Preço Original"] = pd.to_numeric(DB["Preço Original"], errors="coerce")
pod = DB["Preço Original"].map("US$ {:.2f}".format)
poe = DB["Preço Original"].map("€ {:.2f}".format)
poo = DB["Preço Original"].map("{:.2f} g".format)
DB.loc[DB["Moeda"] == "Dólar", "Preço Original"] = pod
DB.loc[DB["Moeda"] == "Euro", "Preço Original"] = poe
DB.loc[DB["Moeda"] == "Ouro", "Preço Original"] = poo

display(DB)

In [ ]:
#Salvando a DB atualizada
nome = "Produtos Atualizados em " + momento + ".xlsx"
DB.to_excel(nome, index=False)